In [1]:
import os
import base64
import pandas as pd
from docx import Document
from PIL import Image
import fitz 
from io import BytesIO
import requests

In [7]:
TOGETHER_API_KEY = "2fb845a832ce73f138c40a606a960ec9146df909af8dc1f0063cb5360c8daaed"
MODEL_ID = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"

headers = {
    "Authorization": f"Bearer {TOGETHER_API_KEY}",
    "Content-Type": "application/json"
}

In [15]:
def extract_text_files(file):
    ext = os.path.splitext(filename)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(file)
        return df.to_string()

    elif ext == ".txt":
        return file.read().decode("utf-8")

    elif ext == ".xls" or ext == ".xlsx":
        df = pd.read_excel(file)
        return df.to_string()

    elif ext == ".docx":
        doc = Document(file)
        return "\n".join([p.text for p in doc.paragraphs])

    elif ext == ".pdf":
        text = ""
        pdf = fitz.open(stream=file.read(), filetype="pdf")
        for page in pdf:
            text += page.get_text()
        return text        

    else:
        return "Error 404"

In [29]:
def extract_image(file):
    return Image.open(BytesIO(file.read())).convert("RGB")

def encode_image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [9]:
def analyze_text(text, question):
    prompt = f"You are a data analyst.\n\n{text}\n\nQuestion: {question}"
    payload = {
        "model": MODEL_ID,
        "prompt": prompt,
        "max_tokens": 512,
        "temperature": 0.7
    }
    response = requests.post("https://api.together.xyz/v1/completions", headers=headers, json=payload)
    return response.json().get("choices", [{}])[0].get("text", "No response from model.")


In [30]:
def analyze_image(image, question):
    img_base64 = encode_image_to_base64(image)
    payload = {
        "model": MODEL_ID,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                ]
            }
        ],
        "max_tokens": 512,
        "temperature": 0.7
    }

    response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response content.")
    else:
        return f"Error: {response.status_code} - {response.text}"


In [20]:
with open("titanic2.csv", "rb") as file:
    text = extract_text_files(file)
    question = "How many rows are there in the attached file"
    result = analyze_text(text, question)
    print("Text Analysis Result:\n", result)

🔎 Text Analysis Result:
 ?
```python
# Import necessary libraries
import pandas as pd

# Assuming the data is in a DataFrame named 'df'
df = pd.read_csv('data.csv')

# Get the number of rows in the DataFrame
num_rows = df.shape[0]

# Print the result
print(num_rows)
```


In [31]:
with open("barchart.jpg", "rb") as file:
    image = extract_image(file)
    question = "Explain the trends shown in this bar chart."
    result = analyze_image(image, question)
    print(result)


The bar chart presents a comparison of sales strategies for four distinct products, labeled A, B, C, and D. The x-axis lists five different strategies, while the y-axis represents the sales figures, ranging from 0 to 200.

**Product Sales Trends:**

*   **Product A (Light Blue):** The sales of Product A remain relatively consistent across all five strategies, with a slight increase observed in Strategy 2 and Strategy 3.
*   **Product B (Dark Blue):** Product B exhibits a notable peak in sales under Strategy 2, followed by a decline in subsequent strategies. Its sales are lowest under Strategies 3, 4, and 5.
*   **Product C (Orange):** Product C consistently demonstrates high sales across all strategies, with a slight increase from Strategy 1 to Strategy 2, and then remains steady.
*   **Product D (Yellow):** Product D shows a significant increase in sales from Strategy 1 to Strategy 2, followed by a plateau in Strategies 3 and 4. However, its sales experience a sharp decline under Stra